# GPT F1 QA Evaluation

This notebook evaluates GPT models (4.1 nano, 4o mini, 5 mini) on the F1 QA dataset with multiple choice questions.

It loads 500 JSON files (1500 QA pairs total), formats them as multiple choice questions, and evaluates the model's accuracy.


In [ ]:
# Install required packages with latest versions
%pip install tqdm


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully!")

import os
from google.colab import userdata
# Check if GPT token is available
if userdata.get('GPT_TOKEN'):
    GPT_TOKEN = userdata.get('GPT_TOKEN')
    print("✅ OpenAI API token available as an environment variable")
else:
    print("⚠️  OpenAI API token not found in environment variables")
    print("   Please set your GPT_TOKEN in Colab secrets")


In [ ]:
# Import utilities with latest transformers syntax
import sys
sys.path.append('/content')
from f1_qa_utils import load_qa_dataset, format_multiple_choice, extract_answer_letter, extract_justification, parse_json_response

from tqdm import tqdm
import json
from pathlib import Path
import time
import gc
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')


## Configuration

Set paths and model to evaluate.


In [ ]:
# Configuration for GPT
MODEL_ID = "gpt-5-mini"
MODEL_NAME = "gpt-5-mini"

DATASET_PATH = "/content/drive/MyDrive/Data_Collection_Code/dataset"
RESULTS_PATH = "/content/drive/MyDrive/CS6220_Project/results"

# Create results directory if needed
Path(RESULTS_PATH).mkdir(parents=True, exist_ok=True)

print(f"✅ Configuration loaded:")
print(f"   Model: {MODEL_NAME}")
print(f"   Model ID: {MODEL_ID}")
print(f"   Dataset: {DATASET_PATH}")
print(f"   Results: {RESULTS_PATH}")
print(f"   📝 Note: Model inference capabilities will be accessed via API key")


## Load QA Dataset


In [ ]:
# Load all QA pairs
qa_dataset = load_qa_dataset(DATASET_PATH)

print(f"📊 Dataset Statistics:")
print(f"   Total QA pairs: {len(qa_dataset)}")
print(f"   Expected: 1500 (500 files × 3 QA pairs each)")

# Show example
if qa_dataset:
    print(f"\n📝 Example QA pair:")
    example = qa_dataset[0]
    print(f"   Question: {example['rephrased_question']}")
    print(f"   Correct Answer: {example['ground_truth_correct_option']}")
    print(f"   Options: {example['options']}")

    # Show example of formatted prompt
    print(f"\n📝 Example formatted prompt:")
    prompt = format_multiple_choice(
        example['rephrased_question'],
        example['options']
    )
    print(f"   Prompt preview: {prompt[:200]}...")


## Model Evaluator Class


In [ ]:
class F1QAEvaluator:
    """Evaluator for F1 QA models with latest transformers syntax."""

    def __init__(self, model_id: str, model_name: str):
        self.model_id = model_id
        self.model_name = model_name

    def generate_response(self, prompt: str, max_new_tokens: int = 250) -> str:
        """Generate response using an API call."""
        try:

            from openai import OpenAI
            #MODEL_NAME = "gpt-5-mini"
            client = OpenAI(api_key=GPT_TOKEN)

            response = client.chat.completions.create(
                model=MODEL_NAME,      # "gpt-4o-mini"
                temperature=1,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            return response.choices[0].message.content.strip()

            # Check if outputs is valid
            if outputs is None or not hasattr(outputs, 'sequences'):
                return ""

            # Decode only the new tokens (excluding the input prompt)
            input_length = inputs["input_ids"].shape[1]
            if len(outputs.sequences[0]) > input_length:
                new_tokens = outputs.sequences[0][input_length:]
                response = self.tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
            else:
                # If no new tokens generated, return empty string
                response = ""

            return response

        except Exception as e:
            print(f"❌ Error generating response: {e}")
            return ""

    def evaluate_dataset(self, qa_dataset: List[Dict]) -> Dict:
        """Evaluate model on entire dataset."""
        results = {
            'model_name': self.model_name,
            'total_questions': len(qa_dataset),
            'correct': 0,
            'incorrect': 0,
            'invalid': 0,
            'accuracy': 0.0,
            'details': []
        }

        print(f"\n🚀 Evaluating {self.model_name} on {len(qa_dataset)} questions...")

        for i, qa_pair in enumerate(tqdm(qa_dataset, desc=f"Evaluating {self.model_name}")):
            question = qa_pair['rephrased_question']
            correct_letter = qa_pair['ground_truth_correct_option']
            options = qa_pair['options']

            # Format as multiple choice
            prompt = format_multiple_choice(question, options)

            # Generate response
            response = self.generate_response(prompt)

            print(f"RESPONSE IS: {response}")


            # Extract answer letter and justification
            predicted_letter = extract_answer_letter(response)
            #justification = extract_justification(response)

            print(f"EXTRACTED ANSWER: {predicted_letter}")

            # Check correctness
            is_correct = predicted_letter == correct_letter
            is_invalid = predicted_letter == ""

            if is_correct:
                results['correct'] += 1
            elif is_invalid:
                results['invalid'] += 1
            else:
                results['incorrect'] += 1

            # Store details
            results['details'].append({
                'question': question,
                'correct_letter': correct_letter,
                'predicted_letter': predicted_letter,
                'is_correct': is_correct,
                'is_invalid': is_invalid,
                'response': response,
                #'justification': justification,
                'prompt': prompt
            })

        # Calculate accuracy
        valid_responses = results['total_questions'] - results['invalid']
        if valid_responses > 0:
            results['accuracy'] = results['correct'] / valid_responses

        print(f"✅ {self.model_name} evaluation complete:")
        print(f"   Correct: {results['correct']}")
        print(f"   Incorrect: {results['incorrect']}")
        print(f"   Invalid: {results['invalid']}")
        print(f"   Accuracy: {results['accuracy']:.2%}")

        return results

    def cleanup(self):
        """Clear model from memory."""
        return


## Run Evaluation


In [ ]:
# Evaluate GPT 4o Mini
print(f"\n{'='*60}")
print(f"🤖 Evaluating Model: {MODEL_NAME}")
print(f"{'='*60}")

evaluator = F1QAEvaluator(MODEL_ID, MODEL_NAME)

results = evaluator.evaluate_dataset(qa_dataset)
evaluator.cleanup()

# Save results to JSON
output_file = Path(RESULTS_PATH) / f"gpt_5_mini_evaluation_results_{int(time.time())}.json"

results_summary = {
    'metadata': {
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'model_name': MODEL_NAME,
        'model_id': MODEL_ID,
        'total_qa_pairs': len(qa_dataset)
    },
    'results': results
}

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results_summary, f, indent=2, ensure_ascii=False)

print(f"\n💾 Results saved to: {output_file}")

# Print summary
print(f"\n{'='*60}")
print(f"📊 GPT 5 Mini EVALUATION SUMMARY")
print(f"{'='*60}")
print(f"   Accuracy: {results['accuracy']:.2%}")
print(f"   Correct: {results['correct']}/{results['total_questions']}")
print(f"   Incorrect: {results['incorrect']}")
print(f"   Invalid: {results['invalid']}")
print(f"\n💾 Full results saved to: {output_file}")
